In [ ]:
from __future__ import print_function, division
from builtins import range, input

import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from datetime import datetime
from sortedcontainers import SortedList

#Veriyi yüklüyoruz
import os
if not os.path.exists('user2movie.json') or \
   not os.path.exists('movie2user.json') or \
   not os.path.exists('usermovie2rating.json') or \
   not os.path.exists('usermovie2rating_test.json'):
   import preprocess2dict


with open('user2movie.json', 'rb') as f:
  user2movie = pickle.load(f)

with open('movie2user.json', 'rb') as f:
  movie2user = pickle.load(f)

with open('usermovie2rating.json', 'rb') as f:
  usermovie2rating = pickle.load(f)

with open('usermovie2rating_test.json', 'rb') as f:
  usermovie2rating_test = pickle.load(f)


N = np.max(list(user2movie.keys())) + 1
# test seti, train veri setinde bulunmayan filmler içerebilir
m1 = np.max(list(movie2user.keys()))
m2 = np.max([m for (u, m), r in usermovie2rating_test.items()])
M = max(m1, m2) + 1
print("N:", N, "M:", M)

if M > 2000:
  print("N =", N, "Devam etmek istediğinizden emin misiniz??")
  print("...")
  exit()


# kullanıcı benzerliklerini bulmak için O(M^2 * N) hesabı yapmalıyız.
# not: w_ij simetrik olduğundan hesaplamaların sadece yarısını yapmalıyız
K = 20 # dikkate almak istediğimiz komşu sayısı
limit = 5 # dikkate almak için kullanıcıların ortak olması gereken ortak film sayısı
neighbors = [] # komşuları bu listede saklayacağız
averages = [] # daha sonra kullanmak üzere her öğenin ortalama puanı
deviations = [] # daha sonra kullanmak üzere her öğenin sapması

for i in range(M):
  i öğesine en yakın K adet öğeyi bulalım
  users_i = movie2user[i]
  users_i_set = set(users_i)

  # Ortalama ve sapmayı hesaplayalım.
  ratings_i = { user:usermovie2rating[(user, i)] for user in users_i }
  avg_i = np.mean(list(ratings_i.values()))
  dev_i = { user:(rating - avg_i) for user, rating in ratings_i.items() }
  dev_i_values = np.array(list(dev_i.values()))
  sigma_i = np.sqrt(dev_i_values.dot(dev_i_values))

  # Daha sonra kullanmak için kaydedelim.
  averages.append(avg_i)
  deviations.append(dev_i)

  sl = SortedList()
  for j in range(M):
    # kendimizi almayacağız o yüzden if ile kontrol edelim
    if j != i:
      users_j = movie2user[j]
      users_j_set = set(users_j)
      common_users = (users_i_set & users_j_set) # kesişim
      if len(common_users) > limit:
        # Ortalama ve sapmayı hesaplıyoruz
        ratings_j = { user:usermovie2rating[(user, j)] for user in users_j }
        avg_j = np.mean(list(ratings_j.values()))
        dev_j = { user:(rating - avg_j) for user, rating in ratings_j.items() }
        dev_j_values = np.array(list(dev_j.values()))
        sigma_j = np.sqrt(dev_j_values.dot(dev_j_values))

        # korelasyon katsayısını hesaplıyoruz
        numerator = sum(dev_i[m]*dev_j[m] for m in common_users)
        w_ij = numerator / (sigma_i * sigma_j)

        # sıralı listeye ekliyoruz ve daha sonra siliyoruz.
        # liste artan olarak sıralandığı için ağırlığı iptal ediyoruz
        # En büyük değer en yakın değer olacak.
        sl.add((-w_ij, j))
        if len(sl) > K:
          del sl[-1]

  # Komşuları depoluyoruz.
  neighbors.append(sl)

  
  if i % 1 == 0:
    print(i)



# Train ve test mse'lerini hesaplamak için komşuları kullanıyoruz.

def predict(i, u):
  # sapmaların ağırlıklı toplamını hesaplıyoruz
  numerator = 0
  denominator = 0
  for neg_w, j in neighbors[i]:
    # ağırlık negatif olarak saklanıyor
    # yani negatif ağırlığın negatifi pozitif ağırlıktır o yüzden - ile çarparak topluyoruz.
    try:
      numerator += -neg_w * deviations[j][u]
      denominator += abs(neg_w)
    except KeyError:
      # komşu aynı kullanıcı tarafından derecelendirilmemiş olabilir
      # sözlüğe iki kere bakmak istemiyoruz.
      # bu yüzden pass ile geçiyoruz.
      pass

  if denominator == 0:
    prediction = averages[i]
  else:
    prediction = numerator / denominator + averages[i]
  prediction = min(5, prediction)
  prediction = max(0.5, prediction) # minimum rating yani puanlama 0.5
  return prediction



train_predictions = []
train_targets = []
for (u, m), target in usermovie2rating.items():
  # bu film için tahmini hesaplayalım
  prediction = predict(m, u)

  # tahmini ve hedefi kaydediyoruz
  train_predictions.append(prediction)
  train_targets.append(target)

test_predictions = []
test_targets = []
# test seti için aynı şeyi yapalım
for (u, m), target in usermovie2rating_test.items():
  # bu film için tahmini hesaplayalım
  prediction = predict(m, u)

  # tekrardan tahmini ve hedefi kaydediyoruz
  test_predictions.append(prediction)
  test_targets.append(target)


# Accuracy hesaplayalım.
def mse(p, t):
  p = np.array(p)
  t = np.array(t)
  return np.mean((p - t)**2)

print('train mse:', mse(train_predictions, train_targets))
print('test mse:', mse(test_predictions, test_targets))


